In [1]:
import pandas as pd

In [2]:
#변형할 파일 submission으로 읽어오기
submission = pd.read_csv('./data/sample_submission.csv')
# submission = pd.read_csv('./submit/넣을파일.csv')

#isbn cold start
train = pd.read_csv('./data/train_ratings.csv')                 #train 값 불러오고

count=train.groupby("isbn").size()                              #isbn별로 묶어서 count
dfcount = pd.DataFrame(count, columns=["count_b"])
train=pd.merge(train,dfcount, how='left', on='isbn')            

submission=pd.merge(submission,dfcount, how='left', on='isbn')  #submission 에 count 생성
submission['count_b'] = submission['count_b'].fillna(0)


submission.set_index("isbn",inplace = True)                                
for row in submission.itertuples():
    if row[3] == 0 :                                            #book count가 되지 않은 곳에
        submission.at[row[0],"rating"] = 6.884027966331795      #1개 매겨진 isbn 의 평균 대입
submission = submission.reset_index()

#user_id cold start
train = pd.read_csv('./data/train_ratings.csv')

count=train.groupby("user_id").size()
dfcount = pd.DataFrame(count, columns=["count_u"])
train=pd.merge(train,dfcount, how='left', on='user_id')

submission=pd.merge(submission,dfcount, how='left', on='user_id')
submission['count_u'] = submission['count_u'].fillna(0)

submission.set_index("user_id",inplace = True)                              
for row in submission.itertuples():
    if row[4] == 0 :                                                       
        submission.at[row[0],"rating"] = 7                       #1명 rating 들의 평균대입
submission = submission.reset_index()


# #count_b, count_u 콜럼 제거
# submission = submission.drop(['count_b'], axis=1)
# submission = submission.drop(['count_u'], axis=1)

In [3]:
count_u = submission['count_u'] ==0 
count_b = submission['count_b'] ==0
user_cold = submission[count_u]
book_cold = submission[count_b]
both_cold= submission[count_b & count_u]
count_b1 = submission['count_b'] != 0
count_u1 = submission['count_u'] != 0
to_train = submission[count_b1 & count_u1]

ori = submission.copy()
submission[count_b & count_u]
print(submission[count_b & count_u])

       user_id        isbn  rating  count_b  count_u
929        176  1900850303     7.0      0.0      0.0
953        190  0002740230     7.0      0.0      0.0
3491       278  0590431110     7.0      0.0      0.0
5123       530  0516076272     7.0      0.0      0.0
5998       705  8806142267     7.0      0.0      0.0
...        ...         ...     ...      ...      ...
76612   275828  0802138748     7.0      0.0      0.0
76623   276067  0694004162     7.0      0.0      0.0
76655   277079  3499263998     7.0      0.0      0.0
76669   277514  3453042476     7.0      0.0      0.0
76679   278012  0140134212     7.0      0.0      0.0

[1734 rows x 5 columns]


In [4]:
ori[count_b & count_u]['rating'] 

929      7.0
953      7.0
3491     7.0
5123     7.0
5998     7.0
        ... 
76612    7.0
76623    7.0
76655    7.0
76669    7.0
76679    7.0
Name: rating, Length: 1734, dtype: float64

ori.loc[count_b & count_u,'rating'] = tmp['rating']



#위 처럼 해당하는 데이터의 값을 할당 시킬 수 있다~~!!!!!!


In [5]:
##############to train 파일은 fm 돌리고 나머지 값들은 인덱싱하자

In [6]:
####### user_cold <- item 있는 거에서 레이팅 평균
####### isbn_cold <- user 있는 거에서 레이팅 평균
####### 둘다 콜드 <- 전체 평균
####### cold start  아닌거 -> fm emb dim 2로 넣어주고 학습시킨거에서 predict

In [7]:
train = pd.read_csv('./data/train_ratings.csv')
#submission.loc[count_b & count_u,'rating'] = tmp['rating']
print(train['rating'].mean())
print(len(train))


7.069714304340032
306795


In [8]:
#book 콜드인 거에서 userid 있는 거의 평균으로 할당
tmp = train[train['user_id'].isin(book_cold['user_id'])]
print(tmp['rating'].mean())
#더했어야함
print(tmp)
book_cold_userid_mean = tmp['rating'].mean()

7.1756075395674


In [9]:
#user 콜드인 거에서 isbn 있는 거의 평균으로 할당
tmp = train[train['isbn'].isin(user_cold['isbn'])]
print(tmp['rating'].mean())
user_cold_isbn_mean = tmp['rating'].mean()

7.233799037019496


In [10]:
#user 콜드인 거에서 isbn 있는 거의 평균으로 할당
tmp = train[train['isbn'].isin(both_cold['isbn'])]
print(tmp)
both_cold_mean = (user_cold_isbn_mean + book_cold_userid_mean)/2
print(both_cold_mean)

Empty DataFrame
Columns: [user_id, isbn, rating]
Index: []
7.204703288293448


In [11]:
#bookcold = count_b
#usercold = count_u
#both = count_b & count_u
submission.loc[count_u,'rating'] = user_cold_isbn_mean
#submission.loc[count_b,'rating'] = book_cold_userid_mean
submission.loc[count_b,'rating'] = 6.884027966331795##1개 매겨진 isbn 의 평균 대입
submission.loc[count_b & count_u,'rating'] = both_cold_mean

cold_submission = submission.copy()


In [12]:
#target = pd.read_csv('./submit/DeepCoNN_52_2.csv')
#target = pd.read_csv('./submit/rule_based_1user.csv')
target = pd.read_csv('./submit/FM_dim2_local_2.4013.csv')
submission['rating'] = target['rating']
submission.loc[count_u,'rating'] = user_cold_isbn_mean
submission.loc[count_b,'rating'] = book_cold_userid_mean
submission.loc[count_b & count_u,'rating'] = both_cold_mean
submission = submission.drop(['count_b'], axis=1)
submission = submission.drop(['count_u'], axis=1)

print(submission)


       user_id        isbn    rating
0        11676  0002005018  7.794324
1       116866  0002005018  9.128166
2       152827  0060973129  6.445954
3       157969  0374157065  7.350306
4        67958  0399135782  8.862909
...        ...         ...       ...
76694   278543  1576734218  7.175608
76695   278563  3492223710  7.175608
76696   278633  1896095186  7.175608
76697   278668  8408044079  7.175608
76698   278851  0767907566  7.175608

[76699 rows x 3 columns]


In [13]:

#submission.to_csv('sj_rulebase_NCF.csv', index=False)
#submission.to_csv('sj_rulebase_1user.csv', index=False)
submission.to_csv('sj_rulebase_FM_2dim_ver2.csv', index=False)